In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score   
import math

%matplotlib inline

In [19]:
sales_train_raw = pd.read_csv('../resources/sales_train_v2.csv')

In [20]:
item_cnt_by_shop_item = sales_train_raw.groupby(['shop_id','item_id','date_block_num'])['item_cnt_day'].agg(sum)

In [21]:
item_cnt_by_shop_item_greater50 = item_cnt_by_shop_item[item_cnt_by_shop_item >= 50]

In [22]:
item_cnt_by_shop_item_greater50 = item_cnt_by_shop_item_greater50.reset_index()

In [23]:
item_cnt_by_shop_item_greater50

,shop_id,item_id,date_block_num,item_cnt_day
0,0,14752,1,67.0
1,0,16482,1,54.0
2,0,19811,1,64.0
3,1,13354,1,54.0
4,2,3731,27,138.0
5,2,3732,8,66.0
6,2,3733,22,87.0
7,2,6502,20,53.0
8,2,7807,16,53.0
9,2,10209,28,73.0


In [24]:
sales_train_raw_out_rem = sales_train_raw[sales_train_raw['']]

KeyError: ''

In [25]:
sales_train = sales_train_raw.copy()

In [26]:
sales_train_0_item = sales_train_raw.copy()

In [27]:
sales_train_0_item['item_cnt_day'] = 0

In [28]:
shop_id_list = sorted(sales_train.shop_id.unique())

In [7]:
# shop IDs : 0-59

In [29]:
shops_excluded = [0, 1, 8, 9, 11, 13, 17, 20, 23, 29, 30, 32, 33, 36, 40, 43, 54]

In [30]:
shop_t = [0] * len(shop_id_list)
for i in shop_id_list:
    shop_t[i] = sales_train[sales_train['shop_id'] == i]

In [31]:
shop = [0] * len(shop_id_list)
for i in shop_id_list:
    shop[i] = [sales_train[sales_train['shop_id'] == i], sales_train_0_item[sales_train['shop_id'] != i]]
    shop[i] = pd.concat(shop[i])

In [11]:
# Shop ID with 0 item for 34th month - 0, 1, 8, 9, 11, 13, 17, 20, 23, 29, 30, 32, 33, 36, 40, 43, 54

In [32]:
shops_excluded = [0, 1, 8, 9, 11, 13, 17, 20, 23, 29, 30, 32, 33, 36, 40, 43, 54]

In [33]:
shop_item_mc = [0] * len(shop_id_list)
for i in shop_id_list:
    shop_item_mc[i] = shop[i].groupby(['date_block_num'])['item_cnt_day'].agg(sum)

In [34]:
shop_ana_df = [0] * len(shop_id_list)
X_shop = [0] * len(shop_id_list)
y_shop = [0] * len(shop_id_list)
lr_shop = [0] * len(shop_id_list)
scores_shop = [0] * len(shop_id_list)
rmse_shop = [0] * len(shop_id_list)
for i in shop_id_list:
    shop_ana_df[i] = shop[i].groupby(['date_block_num','item_id'])['item_cnt_day'].agg(sum).reset_index(['date_block_num','item_id'])
    shop_ana_df[i] = pd.concat([shop_ana_df[i],item_cnt_by_shop_item_greater50[item_cnt_by_shop_item_greater50['shop_id'] == i]['item_id','date_block_num','item_cnt_day']], ignore_index=True)
    shop_ana_df[i].drop_duplicates(subset=['date_block_num','item_id'], keep = False, inplace = True)
    X_shop[i] = shop_ana_df[i][['date_block_num','item_id']]
    X_shop[i]['shop_id'] = i
    y_shop[i] = shop_ana_df[i]['item_cnt_day']
    lr_shop[i] = LinearRegression()
    scores_shop[i] = cross_val_score(lr_shop[i],X_shop[i],y_shop[i],cv=10,scoring="neg_mean_squared_error")
    rmse_shop[i] = math.sqrt(abs(scores_shop[i].mean()))

In [15]:
test_raw = pd.read_csv('../resources/test.csv')

In [36]:
shop_ana_df[1].head()

,date_block_num,item_id,item_cnt_day
0,0,19,0.0
1,0,27,1.0
2,0,28,0.0
3,0,29,1.0
4,0,32,7.0


In [16]:
test = test_raw.copy()

In [17]:
test['date_block_num'] = 34

In [18]:
test_f = test.loc[:,test.columns != 'ID']

In [19]:
pred = [0] * len(shop_id_list)
for i in shop_id_list:
    lr_shop[i].fit(X_shop[i],y_shop[i])
    pred[i] = lr_shop[i].predict(test_f)

In [20]:
pred_f = [0] * len(test_f)
for i in shop_id_list:
    pred_f = pred_f + pred[i]

In [21]:
pred_f = pred_f/len(shop_id_list)

In [22]:
test['item_cnt_month'] = pred_f

In [23]:
test_to_csv = test[['ID','item_cnt_month']]

In [24]:
test_to_csv.to_csv('../outputs/avg_60_shops.csv', index = False)